In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython

import sys,time,datetime,copy,subprocess,itertools,pickle,warnings,json,gc,numbers

import numpy as np
import scipy as sp

from matplotlib import pyplot as plt
import matplotlib as mpl

In [2]:
import numpy

class pywatch():
    
    def __init__(self):
        self.times = []
        
    def record(self):
        self.times.append(time.time())
        return self
    
    def print(self):
        self.times.append( time.time() )
        T = np.array(self.times)
        dT = np.ediff1d(T)
        dT = dT * 1e+3
        print(dT)

In [3]:
num_seq = 3
itv = [0,1000]
T = [ np.random.exponential(size=100).cumsum() for _ in range(5) ]
T = [ T_i[T_i<itv[1]] for T_i in T ]
mark = [ np.ones(T_i.shape[0],dtype='i8') * i for i,T_i in enumerate(T) ]

T = np.hstack(T)
mark = np.hstack(mark)
index_sort = np.argsort(T)
T = T[index_sort]
mark = mark[index_sort]
Data = {'T':T, 'mark':mark}

In [4]:
num_seq = 3
itv = [0,10000]
num_exp = 2
alpha = np.ones((num_exp,num_seq,num_seq)) * 0.1
beta = np.ones((num_exp,num_seq,num_seq))
beta[0] = 10.0
beta[1] = 100.0
mu = np.ones(num_seq)

Data = {'T':[], 'mark':[]}

t = 0
l_sum = mu.sum()
l_trg = np.zeros((num_exp,num_seq,num_seq))

while 1:
    
    l_sum = l_trg.sum() + mu.sum()
    step = np.random.exponential(1/l_sum)
    t += step
    l_trg *= np.exp(-beta*step)
    l_sum_next = l_trg.sum() + mu.sum()
    
    if t > itv[1]:
        break
    
    if np.random.rand() < l_sum_next/l_sum:       
        l_cum = np.cumsum( l_trg.sum(axis=(0,1)) + mu )
        l_cum = l_cum/l_cum[-1]
        index_seq = np.searchsorted(l_cum,np.random.rand())
        Data['T'].append(t)
        Data['mark'].append(index_seq)
        l_trg[:,index_seq,:] += alpha[:,index_seq,:]*beta[:,index_seq,:]
        
Data['T'] = np.array(Data['T'])
Data['mark'] = np.array(Data['mark'])

print(Data['T'].shape[0])

75056


In [5]:
%%cython

import numpy as np
cimport numpy as np

def LG_SUM_kernel_cython(np.ndarray[np.float64_t,ndim=1] T, np.ndarray[np.int64_t,ndim=1] mark, np.ndarray[np.float64_t,ndim=3] alpha, np.ndarray[np.float64_t,ndim=3] beta):
    
    cdef int num_exp = alpha.shape[0]
    cdef int num_seq = alpha.shape[1]
    cdef int num_event = T.shape[0]
    
    cdef np.ndarray[np.float64_t,ndim=1] l    = np.zeros(num_event,dtype=np.float64)
    cdef np.ndarray[np.float64_t,ndim=3] dl_a = np.zeros((num_exp,num_seq,num_event),dtype=np.float64) 
    cdef np.ndarray[np.float64_t,ndim=3] dl_b = np.zeros((num_exp,num_seq,num_event),dtype=np.float64)
    
    cdef np.ndarray[np.float64_t,ndim=3] l_mat    = np.zeros((num_exp,num_seq,num_seq),dtype=np.float64)
    cdef np.ndarray[np.float64_t,ndim=3] dl_mat_a = np.zeros((num_exp,num_seq,num_seq),dtype=np.float64)
    cdef np.ndarray[np.float64_t,ndim=3] dl_mat_b = np.zeros((num_exp,num_seq,num_seq),dtype=np.float64)
    
    cdef int index_exp,index_trg,index_tgt,index_event
    cdef int mark1,mark2
    cdef double step
    
    cdef np.ndarray[np.float64_t,ndim=4] r = np.exp( - beta * np.ediff1d(T).reshape(-1,1,1,1) )
        
    for index_event in range(1,num_event):
        
        mark1 = mark[index_event-1]
        mark2 = mark[index_event]
        step = T[index_event] - T[index_event-1]
        
        for index_exp in range(num_exp):
            for index_tgt in range(num_seq):
                l_mat[   index_exp,mark1,index_tgt] += alpha[index_exp,mark1,index_tgt] * beta[index_exp,mark1,index_tgt]
                dl_mat_a[index_exp,mark1,index_tgt] +=  beta[index_exp,mark1,index_tgt]
                dl_mat_b[index_exp,mark1,index_tgt] += alpha[index_exp,mark1,index_tgt]
            
        for index_exp in range(num_exp):
            for index_trg in range(num_seq):
                for index_tgt in range(num_seq):
                    l_mat[   index_exp,index_trg,index_tgt] *= r[index_event-1,index_exp,index_trg,index_tgt]
                    dl_mat_a[index_exp,index_trg,index_tgt] *= r[index_event-1,index_exp,index_trg,index_tgt]
                    dl_mat_b[index_exp,index_trg,index_tgt] *= r[index_event-1,index_exp,index_trg,index_tgt]
                    dl_mat_b[index_exp,index_trg,index_tgt] -=      step*l_mat[index_exp,index_trg,index_tgt]
                    
        for index_exp in range(num_exp):
            for index_trg in range(num_seq):
                l[index_event] += l_mat[index_exp,index_trg,mark2]
                dl_a[index_exp,index_trg,index_event] = dl_mat_a[index_exp,index_trg,mark2]
                dl_b[index_exp,index_trg,index_event] = dl_mat_b[index_exp,index_trg,mark2]

                
    return [l,{'alpha':dl_a,'beta':dl_b}]

In [6]:
from Quasi_Newton import Quasi_Newton

class estimator():
    
    def __init__(self,num_seq=1):
        self.num_seq = num_seq
    
    def set_kernel(self,num_exp=1):
        self.num_exp = num_exp
        return self
    
    def set_data(self,Data,itv):
        self.Data = Data
        self.itv = itv
        return self
    
    def set_para(self,para):
        num_exp = self.num_exp
        num_seq = self.num_seq
        self.para = para
        self.para['alpha'] = self.para['alpha'].reshape(num_exp,num_seq,num_seq)
        self.para[ 'beta'] = self.para[ 'beta'].reshape(num_exp,num_seq,num_seq)
        return self
    
    def fit(self,Data,itv):
        
        num_exp = self.num_exp
        num_seq = self.num_seq
        
        self.Data = Data
        self.itv = itv
        
        self.stg = {}
        self.stg['para_list'] =      ['mu','alpha','beta']
        self.stg['para_length'] =    {'mu':num_seq, 'alpha':num_exp*num_seq*num_seq, 'beta':num_exp*num_seq*num_seq }
        self.stg['para_exp'] =       {'mu':True, 'alpha':True, 'beta':True }
        self.stg['para_ini'] =       {'mu':np.ones(num_seq)*0.2,  'alpha':np.ones(num_exp*num_seq*num_seq)*0.2,  'beta':np.ones(num_exp*num_seq*num_seq)*1.0 }
        self.stg['para_step_Q'] =    {'mu':np.ones(num_seq)*0.2,  'alpha':np.ones(num_exp*num_seq*num_seq)*0.2,  'beta':np.ones(num_exp*num_seq*num_seq)*0.2 }
        self.stg['para_step_diff'] = {'mu':np.ones(num_seq)*0.01, 'alpha':np.ones(num_exp*num_seq*num_seq)*0.01, 'beta':np.ones(num_exp*num_seq*num_seq)*0.01 }

        [para,L,ste,G_norm,i_loop] = Quasi_Newton(self)
        self.set_para(para)
        
        return para
    
    def LG_baseline(self):
        
        mu = self.para['mu']
        
        T = self.Data['T']
        mark = self.Data['mark']
        itv = self.itv
        st,en = itv
        num_seq = self.num_seq
        num_event = T.shape[0]
        
        l = mu[mark]
        dl_mu = np.ones(num_event)
        Int = mu.sum()*(en-st)
        dInt_mu = np.ones(num_seq)*(en-st)
        
        return [l,{'mu':dl_mu},Int,{'mu':dInt_mu}]
    
    def LG_SUM_kernel(self):
        
        alpha = self.para['alpha']
        beta = self.para['beta']
        
        T = self.Data['T']
        mark = self.Data['mark']
        itv = self.itv
        st,en = itv
        num_exp = self.num_exp
        num_seq = self.num_seq
        num_event = T.shape[0]
        
        l = np.zeros(num_event)
        dl_a = np.zeros((num_exp,num_seq,num_event)) 
        dl_b = np.zeros((num_exp,num_seq,num_event))
        
        l_mat    = np.zeros((num_exp,num_seq,num_seq))
        dl_mat_a = np.zeros((num_exp,num_seq,num_seq))
        dl_mat_b = np.zeros((num_exp,num_seq,num_seq))
        
        for index_event in range(1,num_event):
            
            mark1 = mark[index_event-1]
            mark2 = mark[index_event]
            
            l_mat[   :,mark1,:] += alpha[:,mark1,:] * beta[:,mark1,:]
            dl_mat_a[:,mark1,:] +=  beta[:,mark1,:]
            dl_mat_b[:,mark1,:] += alpha[:,mark1,:]
            
            r = np.exp(-beta*(T[index_event]-T[index_event-1]))
            
            l_mat    *= r
            dl_mat_a *= r
            dl_mat_b  = dl_mat_b * r - l_mat * (T[index_event]-T[index_event-1])
            
            l[index_event] = l_mat[:,:,mark2].sum()
            dl_a[:,:,index_event] = dl_mat_a[:,:,mark2]
            dl_b[:,:,index_event] = dl_mat_b[:,:,mark2]
            
        return [l,{'alpha':dl_a,'beta':dl_b}]

    def LG_INT_kernel(self):
        
        alpha = self.para['alpha']
        beta = self.para['beta']
        
        T = self.Data['T']
        mark = self.Data['mark']
        itv = self.itv
        st,en = itv
        num_exp = self.num_exp
        num_seq = self.num_seq
        num_event = T.shape[0]
        
        t_to_en = (en-T).reshape(1,num_event,1)
        alpha_t = alpha[:,mark,:]
        beta_t  =  beta[:,mark,:]
        mark_t = np.array([ mark == i for i in range(num_seq) ]).astype('f8')
        
        Int = ( alpha_t * ( 1 - np.exp(-beta_t*t_to_en) ) ).sum()
        dInt_a = np.matmul( mark_t, 1 - np.exp(-beta_t*t_to_en) )
        dInt_b = np.matmul( mark_t, alpha_t * np.exp(-beta_t*t_to_en) * t_to_en)
            
        return [Int,{'alpha':dInt_a, 'beta':dInt_b}]
        
    def LG(self,para,only_L=False):
        
        T = self.Data['T']
        mark = self.Data['mark']
        num_seq = self.num_seq
        num_event = T.shape[0]
        
        self.set_para(para)
        
        [l_baseline,dl_baseline,Int_baseline,dInt_baseline] = self.LG_baseline()
        [l_kernel,dl_kernel] = LG_SUM_kernel_cython(T,mark,para['alpha'],para['beta'])
        [Int_kernel,dInt_kernel] = self.LG_INT_kernel()
        
        ### 
        l = l_baseline + l_kernel
        Int = Int_baseline + Int_kernel
        
        mark_t = np.array([ mark == i for i in range(num_seq) ]).astype('f8').transpose()
        
        d_log_l_mu = np.matmul( dl_baseline['mu'] /l, mark_t )
        d_log_l_a  = np.matmul( dl_kernel['alpha']/l, mark_t ) 
        d_log_l_b  = np.matmul( dl_kernel['beta'] /l, mark_t )

        L = np.log(l).sum() - Int
        G_mu = d_log_l_mu - dInt_baseline['mu']
        G_a  = d_log_l_a  - dInt_kernel['alpha']
        G_b  = d_log_l_b  - dInt_kernel['beta']
        
        return [ L, {'mu':G_mu, 'alpha':G_a.flatten(),'beta':G_b.flatten()} ]

In [7]:
model = estimator(num_seq=3).set_kernel(num_exp=2)
model.fit(Data,itv)


{'mu': array([0.99286096, 0.99946419, 0.98487937]),
 'alpha': array([[[0.11072905, 0.10241536, 0.10137293],
         [0.08622819, 0.105433  , 0.0869718 ],
         [0.11498916, 0.09371829, 0.09162092]],
 
        [[0.09800041, 0.09456382, 0.09871877],
         [0.11512904, 0.10019452, 0.10712525],
         [0.09453654, 0.10494356, 0.10327439]]]),
 'beta': array([[[  9.43641005,  94.57442096,   9.55180516],
         [109.09224402,   9.86233689, 104.22993103],
         [ 85.47925663, 114.18710753,   9.21989123]],
 
        [[106.5372304 ,   9.26003907, 100.24214136],
         [ 13.55599887, 103.7342132 ,  10.17295428],
         [  8.35288623,  13.12375625,  90.90055807]]])}